In [1]:
import pandas as pd

In [2]:
market_data = r'database\market_data.csv'
contract = 'au2402'
dtypes = {
    'Contract': str,
    'pre close': float,
    'Pre settle': float,
    'Open': float,
    'High': float,
    'Low': float,
    'Close': float,
    'Settle': float,
    'ch1': float,
    'ch2': float,
    'Volume': int,
    'Amount': float,
    'OI': int,
}

market_data_df = pd.read_csv(market_data, header=0, index_col=1, dtype=dtypes, parse_dates=[
                                 'Date'], date_format='%Y-%m-%d')
contract_data_df = market_data_df.loc[market_data_df['Contract']==contract]
contract_data_df

,Contract,pre close,Pre settle,Open,High,Low,Close,Settle,ch1,ch2,Volume,Amount,OI
Date,,,,,,,,,,,,,
2023-01-17,au2402,419.98,419.98,421.14,421.38,421.14,421.38,421.26,1.40,1.28,2,84.252,2
2023-01-18,au2402,421.38,421.26,421.60,423.34,419.32,419.32,420.88,-1.94,-0.38,6,252.538,8
2023-01-19,au2402,419.32,420.88,422.24,423.22,420.58,422.18,422.06,1.30,1.18,40,1688.258,27
2023-01-20,au2402,422.18,422.06,422.18,426.18,422.18,425.94,424.58,3.88,2.52,31,1316.224,43
2023-01-30,au2402,425.94,424.58,420.08,428.68,420.08,425.36,424.66,0.78,0.08,51,2165.826,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,au2402,483.60,484.18,481.60,482.68,479.86,481.70,481.42,-2.48,-2.76,112640,5422875.176,106807
2024-01-02,au2402,NaN,481.42,481.24,483.80,480.82,483.32,482.40,1.90,0.98,37149,1792116.720,102542
2024-01-03,au2402,NaN,482.40,482.92,483.44,481.10,482.72,482.62,0.32,0.22,56719,2737419.010,88494


In [3]:
def calculate_macd(contract_data_df, fast_period=12, slow_period=26, signal_period=9):
    # 计算EMA（指数移动平均线）
    ema_fast = contract_data_df['Close'].ewm(span=fast_period, adjust=False).mean()
    ema_slow = contract_data_df['Close'].ewm(span=slow_period, adjust=False).mean()

    # 计算MACD线（快慢线之差）
    macd_line = ema_fast - ema_slow

    # 计算信号线（MACD线的EMA）
    signal_line = macd_line.ewm(span=signal_period, adjust=False).mean()

    # 计算MACD柱状图（MACD线与信号线之差）
    macd_histogram = (macd_line - signal_line) * 2

    # 将MACD计算结果连接到合约数据表中
    macd_df =pd.DataFrame({'MACD DIF': macd_line, 'MACD DEA': signal_line, 'MACD M': macd_histogram})
    contract_data_df = pd.merge(contract_data_df, macd_df, on='Date', how='inner')

    return contract_data_df


contract_data_df = calculate_macd(contract_data_df)
contract_data_df

,Contract,pre close,Pre settle,Open,High,Low,Close,Settle,ch1,ch2,Volume,Amount,OI,MACD DIF,MACD DEA,MACD M
Date,,,,,,,,,,,,,,,,
2023-01-17,au2402,419.98,419.98,421.14,421.38,421.14,421.38,421.26,1.40,1.28,2,84.252,2,0.000000,0.000000,0.000000
2023-01-18,au2402,421.38,421.26,421.60,423.34,419.32,419.32,420.88,-1.94,-0.38,6,252.538,8,-0.164330,-0.032866,-0.262929
2023-01-19,au2402,419.32,420.88,422.24,423.22,420.58,422.18,422.06,1.30,1.18,40,1688.258,27,-0.063059,-0.038905,-0.048308
2023-01-20,au2402,422.18,422.06,422.18,426.18,422.18,425.94,424.58,3.88,2.52,31,1316.224,43,0.316947,0.032266,0.569363
2023-01-30,au2402,425.94,424.58,420.08,428.68,420.08,425.36,424.66,0.78,0.08,51,2165.826,59,0.564793,0.138771,0.852043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,au2402,483.60,484.18,481.60,482.68,479.86,481.70,481.42,-2.48,-2.76,112640,5422875.176,106807,2.613716,1.895893,1.435645
2024-01-02,au2402,NaN,481.42,481.24,483.80,480.82,483.32,482.40,1.90,0.98,37149,1792116.720,102542,2.741438,2.065002,1.352873
2024-01-03,au2402,NaN,482.40,482.92,483.44,481.10,482.72,482.62,0.32,0.22,56719,2737419.010,88494,2.762402,2.204482,1.115839


In [3]:
import os

directory = r'database\contract_data'
file_name = contract + '_data.csv'
def save(contract_data_df, ):
    file_path = os.path.join(directory, file_name)
    print('saving', file_name)
    contract_data_df.to_csv(file_path)
save(contract_data_df)

saving au2402_data.csv


In [7]:
file_path = os.path.join(directory, file_name)
try:
    contract_data_df = pd.read_csv(file_path, header=0, index_col=1)
except FileNotFoundError:
    contract_data_df = pd.DataFrame()

In [8]:
contract_data_df

""
